# GIS module

Les opérations GIS font partie intégrantes des opérations réalisées en hydrologie. Cette page 

In [1]:
import geopandas as gpd
import pandas as pd
import warnings
import leafmap
import numpy as np
import xarray as xr


from pathlib import Path

import xdatasets as xd
import xhydro.gis as xhgis
from xhydro.indicators import get_yearly_op

## Creating a map

In [ ]:
m = leafmap.Map(center=(48.63, -74.71), 
                zoom=3,
                basemap="USGS Hydrography")
m

## Watershed delineation

### a) From a list of coordinates

In [ ]:
lng_lat = [(-69.81971, 48.14720), # Lac Saint-Jean watershed
           (-74.393438, 45.572442) # Ottawa river watershed
          ]

### b) From markers on a map

![test](../../docs/_static/_images/example_draw_marker,png)

In [ ]:
# The following code is only useful for the documentation. You should instead remove this code and
# interact with the map as shown above by positionning markers at sites of interest
m.draw_features = [{'type': 'Feature',
  'properties': {},
  'geometry': {'type': 'Point', 'coordinates': [-73.118597, 46.042467]}}] # Richelieu watershed

In [ ]:
%%time

gdf = xhgis.watershed_delineation(coordinates=lng_lat,
                                  map=m)
gdf

In [ ]:
m.zoom_to_gdf(gdf)

### c) From [xdatasets](https://github.com/hydrologie/xdatasets) (Not implemented yet)

This functionality fetches a list of basins from [xdatasets](https://github.com/hydrologie/xdatasets)' supported datasets, and upon request, [xdatasets](https://github.com/hydrologie/xdatasets) provides a `gpd.GeoDataFrame` containing the precalculated boundaries for these basins.

## Extract geographical watershed properties

a) Let's first extract watershed properties

In [ ]:
xhgis.watershed_properties(gdf)

In [ ]:
xhgis.watershed_properties(gdf[['HYBAS_ID', 'geometry']],
                           unique_id='HYBAS_ID',
                           output_format='xarray')

b) Let's then extract climate indicators

In [134]:
# This will be returned by xdatasets ( c) above) eventually
bucket = Path("https://s3.us-east-2.wasabisys.com/watersheds-polygons/MELCC/json")

paths = [
    bucket.joinpath("023003/023003.json"),
    bucket.joinpath("031101/031101.json"),
    bucket.joinpath("040111/040111.json"),
]

gdf = pd.concat([gpd.read_file(path) for path in paths]).reset_index(drop=True)
gdf

,Station,Superficie,geometry
0,023003,208.4591919813271,"POLYGON ((-70.82601 46.81658, -70.82728 46.815..."
1,031101,111.7131058782722,"POLYGON ((-73.98519 45.21072, -73.98795 45.209..."
2,040111,433.440893903503,"POLYGON ((-74.06645 46.02253, -74.06647 46.022..."


In [124]:
datasets = {
    "era5_land_reanalysis": {"variables": ["t2m", "tp", "sd"]},
}
space = {
    "clip": "polygon",  # bbox, point or polygon
    "averaging": True,
    "geometry": gdf,  # 3 polygons
    "unique_id": "Station",
}
time = {
    "timestep": "D",
    "aggregation": {"tp": [np.nansum], "t2m": [np.nanmax, np.nanmin], "sd": [np.nanmean]},
    "start": "1981-01-01",
    "end": "2010-12-31",
    "timezone": "America/Montreal",
}

xds = xd.Query(datasets=datasets, 
               space=space, 
               time=time)



Spatial operations: processing polygon 040111 with era5_land_reanalysis: : 3it [00:00,  5.51it/s]
Temporal operations: processing sd with era5_land_reanalysis: 100%|██████████████████████████████████| 3/3 [00:04<00:00,  1.46s/it]


In [128]:
# This should be what xdatasets returns so users don't have to add it manually
# We should also consider using xarray-pint to improve units conversion 

ds = xds.data.squeeze()
ds["Station"].attrs["cf_role"] = "timeseries_id"
ds['tas'] = ds.t2m - 273.15
ds["tas"].attrs = {"long_name": "2 metre temperature", "units": "C", "standard_name": "air_temperature", "cell_methods": "time: mean"}
ds["tp"].attrs = {"long_name": "Mean daily precipitation flux", "units": "kg m-2 s-1",
                  "standard_name": "precipitation_flux", "cell_methods": "time: mean within days"}

ds["sd"].attrs = {"long_name": "Snow depth", "units": "m",
                  "standard_name": "lwe_thickness_of_surface_snow_amount", "cell_methods": "time: mean within days"}
ds

<xarray.Dataset>
Dimensions:      (time_agg: 4, Station: 3, time: 10957)
Coordinates:
  * time_agg     (time_agg) object 'nanmax' 'nanmean' 'nanmin' 'nansum'
    spatial_agg  <U7 'polygon'
    timestep     <U1 'D'
  * Station      (Station) object '023003' '031101' '040111'
  * time         (time) datetime64[ns] 1981-01-01 1981-01-02 ... 2010-12-31
    source       <U20 'era5_land_reanalysis'
Data variables:
    t2m          (time_agg, Station, time) float64 258.4 264.3 254.1 ... nan nan
    tp           (time_agg, Station, time) float64 nan nan ... 0.004395
    sd           (time_agg, Station, time) float64 nan nan nan ... nan nan nan
    tas          (time_agg, Station, time) float64 -14.76 -8.9 ... nan nan

In [129]:
timeargs = {
    "01": {"month": [1]},
    "02": {"month": [2]},
    "03": {"month": [3]},
    "04": {"month": [4]},
    "05": {"month": [5]},
    "06": {"month": [6]},
    "07": {"month": [7]},
    "08": {"month": [8]},
    "09": {"month": [9]},
    "10": {"month": [10]},
    "11": {"month": [11]},
    "12": {"month": [12]},
    "spring": {"date_bounds": ["02-11", "06-19"]},
    "summer_fall": {"date_bounds": ["06-20", "11-19"]},
    "year": {"date_bounds": ["01-01", "12-31"]},
    }



In [130]:
ds_vars = {
    "tasmax": (
        "mean",
        ds[["tas"]]
        .sel(time_agg="nanmax")
        .rename({"tas": "tasmax"})
        .squeeze()
        .drop("time_agg"),
    ),
    "tasmin": (
        "mean",
        ds[["tas"]]
        .sel(time_agg="nanmin")
        .rename({"tas": "tasmin"})
        .squeeze()
        .drop("time_agg"),
    ),
    "tasmax": (
        "mean",
        ds[["tas"]]
        .sel(time_agg="nanmax")
        .rename({"tas": "tasmax"})
        .squeeze()
        .drop("time_agg"),
    ),
    "tp": (
        "sum", 
        ds[["tp"]]
        .sel(time_agg="nansum")
        .squeeze()
        .drop("time_agg")
    ),
    "sd": (
        "mean", 
        ds[["sd"]]
        .sel(time_agg="nanmean")
        .squeeze()
        .drop("time_agg")
    ),
}

ds_climatology = xr.merge(
    [
        get_yearly_op(value[1], input_var=key, op=value[0], timeargs=timeargs)
        for (key, value) in ds_vars.items()
    ]
)
ds_climatology

<xarray.Dataset>
Dimensions:                  (Station: 3, time: 30)
Coordinates:
    spatial_agg              <U7 'polygon'
    timestep                 <U1 'D'
  * Station                  (Station) object '023003' '031101' '040111'
    source                   <U20 'era5_land_reanalysis'
  * time                     (time) datetime64[ns] 1981-01-01 ... 2010-01-01
Data variables: (12/60)
    tasmax_mean_01           (Station, time) float64 -11.77 -11.53 ... -5.126
    tasmax_mean_02           (Station, time) float64 0.9956 -8.116 ... -4.01
    tasmax_mean_03           (Station, time) float64 1.001 0.04047 ... 4.128
    tasmax_mean_04           (Station, time) float64 9.624 5.138 ... 8.063 11.01
    tasmax_mean_05           (Station, time) float64 16.73 17.06 ... 13.93 18.27
    tasmax_mean_06           (Station, time) float64 21.22 20.04 ... 19.56 19.8
    ...                       ...
    sd_mean_10               (Station, time) float64 0.0005241 ... 0.0002506
    sd_mean_11               (Station, time) float64 0.01119 ... 0.006283
    sd_mean_12               (Station, time) float64 0.07727 0.0101 ... 0.03509
    sd_mean_spring           (Station, time) float64 0.02299 0.1608 ... 0.048
    sd_mean_summer_fall      (Station, time) float64 0.0002876 ... 0.0007904
    sd_mean_year             (Station, time) float64 0.02809 0.07815 ... 0.03023
Attributes:
    cat:variable:          ('tasmax_mean_01',)
    cat:xrfreq:            AS-JAN
    cat:frequency:         yr
    cat:processing_level:  indicators
    cat:id:

In [133]:
pd.set_option('display.max_rows', 500)
ds_climatology.mean('time').to_dataframe().T

Station,023003,031101,040111
spatial_agg,polygon,polygon,polygon
timestep,D,D,D
source,era5_land_reanalysis,era5_land_reanalysis,era5_land_reanalysis
tasmax_mean_01,-7.658667,-4.886919,-8.37699
tasmax_mean_02,-5.234553,-2.926413,-5.985243
tasmax_mean_03,0.420012,2.771163,-0.403541
tasmax_mean_04,7.987893,10.723399,7.383795
tasmax_mean_05,16.17139,17.831226,15.529333
tasmax_mean_06,21.194927,22.952821,20.460677
tasmax_mean_07,23.291424,25.342737,22.490552
